<a href="https://colab.research.google.com/github/emkafie/Machine-Learning/blob/main/P1_JS14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Praktikum 1**

In [4]:
import zipfile
import os
import shutil

local_zip = 'dataset.zip' # Pastikan nama file zip sesuai
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/dataset_local') # Ekstrak ke folder baru
zip_ref.close()

print("Dataset berhasil diekstrak ke /content/dataset_local")

Dataset berhasil diekstrak ke /content/dataset_local


## **Langkah 1 & 2: Import Library dan Pra-Pengolahan Data**

Pada tahap ini, ImageDataGenerator digunakan untuk melakukan augmentasi data (memperbanyak variasi data latih secara artifisial) guna mencegah overfitting.

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image

# 1. Pra-Pengolahan Data Training
# Rescale: Normalisasi piksel dari 0-255 menjadi 0-1
# Shear, Zoom, Flip: Augmentasi geometri untuk variasi data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Memuat data training dari direktori
print("Memuat Data Training...")
training_set = train_datagen.flow_from_directory('/content/dataset_local/dataset/test_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

# 2. Pra-Pengolahan Data Testing
# Hanya dilakukan rescale (normalisasi), tidak boleh ada augmentasi geometri
test_datagen = ImageDataGenerator(rescale = 1./255)

# Memuat data testing dari direktori
print("Memuat Data Testing...")
test_set = test_datagen.flow_from_directory('/content/dataset_local/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Memuat Data Training...
Found 2000 images belonging to 2 classes.
Memuat Data Testing...
Found 2000 images belonging to 2 classes.


Found X images belonging to 2 classes: Menandakan bahwa direktori terbaca dengan benar. Sistem secara otomatis mendeteksi dua sub-folder (biasanya folder cats dan dogs) sebagai dua kelas yang berbeda (0 dan 1).

## **Langkah 3: Pembuatan Model CNN**

In [7]:
# Inisiasi Model Sequential
cnn = tf.keras.models.Sequential()

# Layer Konvolusi 1 & Pooling 1
# Input shape disesuaikan dengan target_size (64, 64, 3 channel warna RGB)
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Layer Konvolusi 2 & Pooling 2
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Flattening (Mengubah matriks 2D menjadi vektor 1D)
cnn.add(tf.keras.layers.Flatten())

# Fully Connected Layer (Hidden Layer)
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Output Layer
# Units=1 dan activation='sigmoid' karena klasifikasi biner (0 atau 1)
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile Model
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Menampilkan ringkasan arsitektur model
cnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 29, 29, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 813,217 (3.10 MB)

 Trainable params: 813,217 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

* Layer (type): Jenis layer yang digunakan (Conv2D, MaxPooling2D, Flatten, Dense).

* Output Shape: Dimensi data yang keluar dari setiap layer. Perhatikan bagaimana dimensi gambar mengecil setelah Pooling (dari 64x64 menjadi 32x32, lalu 15x15), namun dimensi vektor memanjang setelah Flatten.

* Param #: Jumlah parameter (bobot) yang akan dilatih.

## **Langkah 4: Pelatihan Model (Fitting)**

In [8]:
print("\n--- Mulai Pelatihan Model ---")
# Melatih model dengan data training dan memvalidasi dengan data testing
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 25)


--- Mulai Pelatihan Model ---
Epoch 1/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 164ms/step - accuracy: 0.4866 - loss: 0.7300 - val_accuracy: 0.5315 - val_loss: 0.6903
Epoch 2/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 108ms/step - accuracy: 0.5503 - loss: 0.6931 - val_accuracy: 0.5765 - val_loss: 0.6895
Epoch 3/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 122ms/step - accuracy: 0.5439 - loss: 0.6849 - val_accuracy: 0.5740 - val_loss: 0.6672
Epoch 4/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 154ms/step - accuracy: 0.5875 - loss: 0.6648 - val_accuracy: 0.6320 - val_loss: 0.6462
Epoch 5/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 106ms/step - accuracy: 0.6242 - loss: 0.6388 - val_accuracy: 0.6725 - val_loss: 0.6190
Epoch 6/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.6646 - loss: 0.6198 - val_accuracy: 0.6775 - val_loss: 0.5974
Epoch 7/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 107ms/step - accuracy: 0.6842 - loss: 0.5941 - val_accuracy: 0.6975 - val_loss: 0.5854
Epoch 8/25
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 130ms/step - accuracy: 0.6767 

## **Langkah 5: Prediksi Citra Tunggal**

In [14]:
image_path = '/content/dataset_local/dataset/single_prediction/cat_or_dog_1.jpg'

try:
    # Memuat gambar dan mengubah ukuran ke 64x64 (sesuai input model)
    test_image = image.load_img(image_path, target_size = (64, 64))

    # Mengubah gambar menjadi array numpy
    test_image_array = image.img_to_array(test_image)

    # Menambah dimensi batch (karena model menerima input batch, bukan single image)
    # Dari (64, 64, 3) menjadi (1, 64, 64, 3)
    test_image_expanded = np.expand_dims(test_image_array, axis = 0)

    # Melakukan prediksi
    result = cnn.predict(test_image_expanded)

    # Mengecek indeks kelas (misal: {'cat': 0, 'dog': 1})
    print(f"Indeks Kelas: {training_set.class_indices}")

    # Logika penentuan hasil
    # Karena sigmoid outputnya 0 s.d 1. Biasanya > 0.5 dianggap 1.
    if result[0][0] == 1:
        prediction = 'dog'
    else:
        prediction = 'cat'

    print(f"Hasil Prediksi Raw: {result[0][0]}")
    print(f"Prediksi Akhir: {prediction}")

except FileNotFoundError:
    print(f"File gambar tidak ditemukan di path: {image_path}. Pastikan file sudah ada.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step
Indeks Kelas: {'cats': 0, 'dogs': 1}
Hasil Prediksi Raw: 1.0
Prediksi Akhir: dog


Hasil Prediksi Benar

* Indeks Kelas: Menampilkan dictionary mapping label folder ke angka, contoh: {'cats': 0, 'dogs': 1}. Ini menjadi acuan hasil prediksi.

* Hasil Prediksi Raw: Angka float output dari fungsi aktivasi Sigmoid (antara 0 dan 1).

* Prediksi Akhir: Label teks ('dog' atau 'cat') berdasarkan nilai raw dan indeks kelas. Jika hasil raw mendekati 1, maka diklasifikasikan sebagai anjing (sesuai indeks 'dogs': 1), dan sebaliknya.